# Imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F


# Data Loading

In [ ]:
# from torch.utils.data import Dataset

# class imageDataset(Dataset):
#     #Subclass datasets in order to create a custom one that i dont have to load everything in till im ready
    
#     #All neeccesary methods to implement: Inits it with data
#     def __init__(self, folder_path):
#         self.data = #Stuff from folder path

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         #Gets the item at idx in form (data, target label) or (X, Y)

In [103]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()]
)

data_dir = "data/cloud_data" 
dataset = datasets.ImageFolder(root=data_dir, transform=transform) #Note. Add a transforms=transforms

train_data, test_data, val_data = random_split(dataset, [0.75, .15, .10])

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True) #prolly dont have to shuffle these 2 but might as well

human_labels = ["Altocumulus", "Altostratus", "Cumulonimbus","Cirrocumulus", "Cirrus", "Cirrostratus", "Contrail", "Cumulus",
                 "Nimbus", "Stratocumulus", "Stratus" ] #human readable labels

In [ ]:
print(type(dataset))

<class 'torchvision.datasets.folder.ImageFolder'>


In [ ]:
for data in dataloader:
    # print(torch.squeeze(data[0][0]).shape) #Data appears to be from 0 to 1
    img = transforms.ToPILImage()(torch.squeeze(data[0][0]))
    
    img.show()
    break


torch.Size([3, 256, 256])


# Basic CNN (no transfer learning)

In [88]:
class convNet(nn.Module):
    def __init__(self):
        super().__init__()
        #nn.conv2d -> in_channels, out_channels, kernel_size
        #Stride defaults to 1
        #Halves on wxh which is done 2x.
        self.all_pooling = nn.MaxPool2d(2, 2)

        #After this: 256-4 -> 252
        self.conv1 = nn.Conv2d(3, 6, 5) #Conv layer 1. Padding is 0

        #Decreases by another 4 
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.conv3 = nn.Conv2d(16, 32, 5)

        self.conv_layers = [self.conv1, self.conv2, self.conv3] #Put all conv layers and fcs into a list for ease of iteration through later?

        self.flatten = nn.Flatten()
        
        flattened_size = 25088
        self.fc1 = nn.Linear(flattened_size, 120)
        self.fc2 = nn.Linear(120, 180)
        self.fcs = [self.fc1, self.fc2]
        self.final_fc = nn.Linear(180, 11)


    def forward(self, x):
        for conv_layer in self.conv_layers:
            x = self.all_pooling(F.relu(conv_layer(x)))
        # x = self.all_pooling(F.relu(self.conv1(x)))
        # x = self.all_pooling(F.relu(self.conv2(x)))    
        x = self.flatten(x)

        for fc in self.fcs:
            x = F.relu(fc(x))
        x = self.final_fc(x)

        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x)) 
        return x

In [89]:
model  = convNet()
for data in dataloader:
    model.forward(data[0])
    break


In [90]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() #Multi class loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)



In [106]:
def eval_model(model, test_loader):
    running_acc = 0

    for inputs, labels in test_loader:
        outputs = model(inputs)
        running_acc += torch.sum(torch.argmax(outputs, dim = 1) == labels) / len(labels)

    return running_acc / len(test_loader)

In [104]:
batches_to_print = 20
print("starting training")
for epoch in range(50): 

    running_loss = 0
    running_acc = 0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        model.train() #if we ever decide to batchnorm later. jujst tells it that we're in training mode rn
        optimizer.zero_grad() #zero grad
        
        outputs = model(inputs) #fd it
 
        acc = torch.sum(torch.argmax(outputs, dim = 1) == labels) / len(labels)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        running_acc += acc.item()
        running_loss += loss.item()

        if i % batches_to_print == batches_to_print - 1:    # print every 20 batches
            print(f'Epoch {epoch + 1} batcj {i + 1} loss: {running_loss / batches_to_print:.3f} acc {running_acc / batches_to_print:.3f}')
            running_acc = 0
            running_loss = 0 

print('Finished Training') 

starting training
Epoch 1 batcj 20 loss: 0.183 acc 0.928
Epoch 1 batcj 40 loss: 0.177 acc 0.931
Epoch 1 batcj 60 loss: 0.221 acc 0.906
Epoch 2 batcj 20 loss: 0.124 acc 0.952
Epoch 2 batcj 40 loss: 0.137 acc 0.950
Epoch 2 batcj 60 loss: 0.203 acc 0.912
Epoch 3 batcj 20 loss: 0.131 acc 0.931
Epoch 3 batcj 40 loss: 0.102 acc 0.950


KeyboardInterrupt: 

In [ ]:
#what the f*** how is it 93 acc when i just threw some random stuff together???
print(eval_model(model, test_loader))

tensor(0.9291)


In [ ]:
torch.save(model, "clouds")

# Resnet